Get the Cookies and headers More like a people.

In [14]:
from seleniumwire import webdriver  
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

# - step1 selenium 上網登入 ee-class
url = "https://w2.math.ncu.edu.tw/" # = ee-class url

# : user agent setting
ua = UserAgent()
user_agent = ua.random

# : option setting
chrome_opt = Options()
chrome_opt.add_argument("start-maximized")  # 設定瀏覽器啟動時開啟視窗最大化，提供較大的視野和操作空間
chrome_opt.add_argument("--disable-proxy-certificate-handler")  # 禁用代理伺服器的證書處理，避免因證書問題而影響網頁載入
chrome_opt.add_argument("--disable-content-security-policy")  # 禁用內容安全政策(CSP)，允許加載非同源的腳本，有助於測試開發
chrome_opt.add_argument("--ignore-certificate-errors")  # 忽略證書錯誤，有助於訪問自簽名證書的開發環境或測試站點
chrome_opt.add_argument("--incognito")  # 啟動無痕(隱身)模式，不會保存瀏覽歷史、Cookies等資料，有助於保護隱私和測試

# : driver setting 
driver = webdriver.Chrome(options = chrome_opt) # => insert the option to the driver (can be overridden)
driver.delete_all_cookies() # 防止污染
driver.get(url)
sleep(2)

# : driver get the title, content, links
driver_title = driver.title
driver_content = driver.find_element(By.TAG_NAME, "body").text
driver_all_links_elements = driver.find_elements(By.XPATH, "//a[@href]")

# : Get the Cookies & Headers
current_login_cookie = driver.get_cookies()
current_login_requests = driver.requests

headers = {}
for request in current_login_requests:
    if request.url == url:
        headers = {k: v for k, v in request.headers.items()}

print("Cookies:", current_login_cookie)
print("Headers:", headers)

sleep(5)
driver.quit()

Cookies: [{'domain': '.ncu.edu.tw', 'expiry': 1710658194, 'httpOnly': False, 'name': '_gat_gtag_UA_153670547_1', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': '1'}, {'domain': 'w2.math.ncu.edu.tw', 'expiry': 1710665329, 'httpOnly': False, 'name': 'XSRF-TOKEN', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'eyJpdiI6IjVlWlpqWmRhbHZXS2ZoWDNvaDl4QlE9PSIsInZhbHVlIjoiUjQ5bGxLdm84S3pBZFlrUW16ZGs4VXJUZWZZdHFkZnVjc1ZxTEVLcFwvMFFaenlxcTZKNjNtTjljalwvN0JrdWtVIiwibWFjIjoiNjFkMTNkYjI2N2Y0ZjRiZjg4MWE5ZTE1MGI4MWFjNTZiZmMyZWRhZjQ5YThjNzkxN2JmZTA0YzVlMDY0ZDZkYiJ9'}, {'domain': '.ncu.edu.tw', 'expiry': 1745218134, 'httpOnly': False, 'name': '_ga', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'GA1.3.1952377605.1710658134'}, {'domain': 'w2.math.ncu.edu.tw', 'expiry': 1710665329, 'httpOnly': True, 'name': 'laravel_session', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'eyJpdiI6ImFyWEJVK1l5eHVqaVNWQzJSdm96UXc9PSIsInZhbHVlIjoiNVwvN0kyVGJzR0xqbHlNNWp4WXZ1TVBZO

'國立中央大學數學系\n公告\n學術活動\n系所資訊\n系所成員\n課程介紹\n入學資訊\n活動紀錄\n立即捐款\n系PORTAL\n成績查詢\nLANGUAGE\n國立中央大學數學系\n數學科學組 計算與資料科學組\n置頂公告\n1122數學系研究生學位考試申請須知\n2024/02/21\n查看更多\n最新消息\n1122數學系研究生學位考試申請須知\n2024/02/21\n校慶系列講座第一場(2/27)：探索數學中的對稱之美\n2024/02/15\n校慶系列講座第三場(5/7)：「數學的魅力：從自然到科技」\n2024/02/15\n系友演講〖AI時代的求職力〗\n2023/11/15\n113學年度數學系碩士班甄試得參加複試名單及口試通知\n2023/10/27\n113學年度數學系特殊選才得參加複試名單\n2023/10/27\n113碩士班甄試9/27開始報名\n2023/09/26\n查看更多\n學術活動\nGaudin Hamiltonians on unitarizable modules over classical Lie (super)algebras\n2024/03/07\nGeometry and Topology in Mathematical AI\n2024/02/29\nTits alternative and its application\n2024/02/22\nEvolution of Diffusion Models: From Birth to Enhanced Efficiency and Controllability\n2024/02/22\nEarly Detection and Classification of Liver Fibrosis with Machine Learning Methods: Image-Based Biomarkers and Data-driven Computational Techniques in Dynamic Contrast-Enhanced MRI\n2024/01/09\nVariational Principles in Fluid Mechanics\n2023/12/21\nOn the newform theory for p-adic class

Request the website Content

In [5]:
import requests
from bs4 import BeautifulSoup
import re

try:
    with requests.Session() as session: 
    # ~ Session can update the performance of the server. 
    
        # : Update the Cookies & Headers from driver
        for cookie in current_login_cookie:
            session.cookies.set(cookie['name'], cookie['value'])
        session.headers.update(headers)
    
        # :  Get the response of the website.
        resp = session.get(url)
        
        # @ Check the response and Parser the response to HTML.
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        
        # @ Get the elements of the site.
        title = soup.title.get_text() if soup.title else ''
        
        content = soup.get_text().strip()
        content = re.sub(r'\n+', '\n', content)
        
        links = []
        all_links = soup.find_all(href=True)
        for link in all_links:
            href = link.get('href') 
            text = link.get_text().strip()

            if href:
                
                # : Get the Link and Href from the link
                if "http" not in href: 
                    href = f"{url}/{href}" if href[0] != "/" else f"{url}{href}" # = href that include the relative path
                
                if text == "": 
                    text = link.get('title','').strip()
                
                links.append((text,href))
                
    print(title)
    print(content)
            
except requests.HTTPError as e:
    print('HTTP error', e.response.status_code)
except requests.RequestException as e:
    print('Request error:', e)
except Exception as e:
    print('', '', [])

國立中央大學數學系
國立中央大學數學系
 
            國立中央大學數學系
        
公告
重要公告
最新消息
歷史公告
獎學金
學術活動
演講訊息
研討會公告
歷史紀錄
系所資訊
系所簡介
數學計算實驗室
機器人實驗室
圖書室
研究計畫
歷屆系主任
大事紀要
聯絡資訊
系友會
系所成員
專任教師
兼任教師
博士後研究員
訪問學者
行政人員
退休教師
課程介紹
修課規定
課程簡介
課程地圖
開放式課程
核心能力表
入學資訊
數學科學組
計算與資料科學組
學士班入學
碩士班入學
博士班入學
活動紀錄
演講紀錄
親師交流
學生活動
研討會紀錄
其他
立即捐款
系Portal
成績查詢
Language
English
Indonesian
Vietnamese
 國立中央大學數學系  
數學科學組
 計算與資料科學組
 置頂公告 
1122數學系研究生學位考試申請須知
2024/02/21
 查看更多 
 最新消息 
1122數學系研究生學位考試申請須知
2024/02/21
校慶系列講座第一場(2/27)：探索數學中的對稱之美
2024/02/15
校慶系列講座第三場(5/7)：「數學的魅力：從自然到科技」
2024/02/15
系友演講〖AI時代的求職力〗
2023/11/15
113學年度數學系碩士班甄試得參加複試名單及口試通知
2023/10/27
113學年度數學系特殊選才得參加複試名單
2023/10/27
113碩士班甄試9/27開始報名
2023/09/26
 查看更多 
 學術活動 
Gaudin Hamiltonians on unitarizable modules over classical Lie (super)algebras
2024/03/07
Geometry and Topology in Mathematical AI
2024/02/29
Tits alternative and its application
2024/02/22
Evolution of Diffusion Models: From Birth to Enhanced Efficiency and Controllability
2024/02/22
Early Detection and Classification of Liver 

In [9]:
with requests.Session() as session: 
# ~ Session can update the performance of the server. 

    # : Update the Cookies & Headers from driver
    for cookie in current_login_cookie:
        session.cookies.set(cookie['name'], cookie['value'])
    session.headers.update(headers)

    # :  Get the response of the website.
    resp = session.get(url)
    
    # @ Check the response and Parser the response to HTML.
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    
    # @ Get the elements of the site.
    title = soup.title.get_text() if soup.title else ''
    
    content = soup.get_text().strip()
    content = re.sub(r'\n+', '\n', content)
    
content

'國立中央大學數學系\n \n            國立中央大學數學系\n        \n公告\n重要公告\n最新消息\n歷史公告\n獎學金\n學術活動\n演講訊息\n研討會公告\n歷史紀錄\n系所資訊\n系所簡介\n數學計算實驗室\n機器人實驗室\n圖書室\n研究計畫\n歷屆系主任\n大事紀要\n聯絡資訊\n系友會\n系所成員\n專任教師\n兼任教師\n博士後研究員\n訪問學者\n行政人員\n退休教師\n課程介紹\n修課規定\n課程簡介\n課程地圖\n開放式課程\n核心能力表\n入學資訊\n數學科學組\n計算與資料科學組\n學士班入學\n碩士班入學\n博士班入學\n活動紀錄\n演講紀錄\n親師交流\n學生活動\n研討會紀錄\n其他\n立即捐款\n系Portal\n成績查詢\nLanguage\nEnglish\nIndonesian\nVietnamese\n 國立中央大學數學系  \n數學科學組\n 計算與資料科學組\n 置頂公告 \n1122數學系研究生學位考試申請須知\n2024/02/21\n 查看更多 \n 最新消息 \n1122數學系研究生學位考試申請須知\n2024/02/21\n校慶系列講座第一場(2/27)：探索數學中的對稱之美\n2024/02/15\n校慶系列講座第三場(5/7)：「數學的魅力：從自然到科技」\n2024/02/15\n系友演講〖AI時代的求職力〗\n2023/11/15\n113學年度數學系碩士班甄試得參加複試名單及口試通知\n2023/10/27\n113學年度數學系特殊選才得參加複試名單\n2023/10/27\n113碩士班甄試9/27開始報名\n2023/09/26\n 查看更多 \n 學術活動 \nGaudin Hamiltonians on unitarizable modules over classical Lie (super)algebras\n2024/03/07\nGeometry and Topology in Mathematical AI\n2024/02/29\nTits alternative and its application\n2024/02/22\nEvolution of Diffusion Models: From Birth to Enhanced Efficiency

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# 啟動 Chrome 瀏覽器。
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 訪問指定的 URL。
driver.get("https://w2.math.ncu.edu.tw//member/full")

# 等待一段時間，讓網頁完全加載。
time.sleep(5)  # 等待 5 秒，根據實際情況調整。

# 獲取網頁標題和渲染後的內容。
title = driver.title
content = driver.find_element(By.TAG_NAME, "body").text

sleep(2)

# 獲取網頁上所有的連結
all_links_elements = driver.find_elements(By.XPATH, "//a[@href]")

links = []
for element in all_links_elements:
    href = element.get_attribute('href')
    text = element.text.strip()

    if href:
        # 確保連結是完整的 URL
        if "http" not in href:
            href = f"{url}/{href}" if href[0] != "/" else f"{url}{href}"

        if text == "":
            # 如果連結文本為空，嘗試獲取 title 屬性
            text = element.get_attribute('title').strip()

        links.append((text, href))

# 顯示所有連結
for text, href in links:
    print(f"Text: {text}, Href: {href}")

print(title)
print(content)

# 關閉瀏覽器。
driver.quit()

Text: 國立中央大學數學系, Href: https://w2.math.ncu.edu.tw/
Text: 公告, Href: https://w2.math.ncu.edu.tw//member/full#
Text: , Href: https://w2.math.ncu.edu.tw/annou/top
Text: , Href: https://w2.math.ncu.edu.tw/annou/news
Text: , Href: https://w2.math.ncu.edu.tw/annou/history
Text: , Href: https://w2.math.ncu.edu.tw/annou/scholarship
Text: 學術活動, Href: https://w2.math.ncu.edu.tw//member/full#
Text: , Href: https://w2.math.ncu.edu.tw/academic/speeches
Text: , Href: https://w2.math.ncu.edu.tw/academic/seminars
Text: , Href: https://w2.math.ncu.edu.tw/academic/history
Text: 系所資訊, Href: https://w2.math.ncu.edu.tw//member/full#
Text: , Href: https://w2.math.ncu.edu.tw/about/info
Text: , Href: https://w2.math.ncu.edu.tw/about/mcl
Text: , Href: https://w2.math.ncu.edu.tw/about/aimakerlab
Text: , Href: https://w2.math.ncu.edu.tw/about/library
Text: , Href: https://w2.math.ncu.edu.tw/about/plan
Text: , Href: https://w2.math.ncu.edu.tw/about/chairman
Text: , Href: https://w2.math.ncu.edu.tw/about/event
Text

In [16]:
import json

# Load dataset
def read_data(data_name):
    with open(data_name, 'r' ,encoding='utf-8') as f:
        data = json.load(f)
        for row in data:
            yield row
            
read_data("result.json")

<generator object read_data at 0x117000f40>

In [17]:
temp_list = []
with open("result.json", 'r' ,encoding='utf-8') as f:
    data = json.load(f)
    for row in data:  
        temp_list.append(row)
        
len(temp_list)

160

In [20]:
from gensim.models import Word2Vec
import numpy as np

# 假設model是你的Word2Vec模型
model = Word2Vec.load("./d2v_model")

def word2vec_transform(text):
    """
    將文本轉換為向量的函數。
    文本被分割成單詞，每個單詞的向量表示被獲取並平均，來表示整個文本。
    """
    # 分割文本為單詞
    words = text.split()
    
    # 初始化一個空的向量
    vector_sum = np.zeros(model.vector_size)
    
    # 計算包含的單詞的數量
    num_words = 0
    
    # 對於文本中的每個單詞，如果該單詞在模型的詞彙中，加上它的向量表示
    for word in words:
        if word in model.wv:
            vector_sum += model.wv[word]
            num_words += 1
    
    # 如果文本中有模型詞彙表中的單詞，返回平均向量；否則返回零向量
    if num_words > 0:
        vector_average = vector_sum / num_words
        return vector_average
    else:
        # 如果文本中沒有任何單詞在詞彙表中，返回一個零向量
        return np.zeros(model.vector_size)

# 使用示例
text = "這是一個例子文本"
vector_representation_of_text = word2vec_transform(text)
print(vector_representation_of_text)


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [2]:
from elasticsearch import Elasticsearch
import json

# 連接到Elasticsearch
es = Elasticsearch(hosts=["http://localhost:9200"])

# 執行查詢
res = es.search(index="ncu_lrn", body={"query": {"match_all": {}}})

# 將結果轉換為字典（如果需要的話，可以進一步處理這些數據）
results = [doc["_source"] for doc in res['hits']['hits']]

# 將結果存檔為JSON
with open('es_results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("結果已存檔為 es_results.json")


結果已存檔為 es_results.json


In [ ]:

from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

In [1]:
import json
from graphviz import Digraph


# 讀取 JSON 檔案
with open('result.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 創建 Digraph 對象
dot = Digraph(comment='Sitemap')

# 為每個頁面添加節點和邊
for page in data:
    dot.node(page['url'], page['title'])
    for link in page['links']:
        dot.edge(page['url'], link[1], label=link[0])

# 保存和/或顯示圖形
dot.render('sitemap.gv', view=True)


Error: sitemap.gv: syntax error in line 2085 near '.'


CalledProcessError: Command '[WindowsPath('dot'), '-Kdot', '-Tpdf', '-O', 'sitemap.gv']' returned non-zero exit status 1. [stderr: b"Error: sitemap.gv: syntax error in line 2085 near '.'\r\n"]

In [22]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# 載入模型和分詞器
model_names = [  "sentence-transformers/paraphrase-mpnet-base-v2",
                "sentence-transformers/multi-qa-mpnet-base-dot-v1", 
                "sentence-transformers/multi-qa-MiniLM-L6-cos-v1",
                "sentence-transformers/all-MiniLM-L12-v2",
                "sentence-transformers/all-MiniLM-L6-v2",
                "sentence-transformers/all-mpnet-base-v2",
                "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
                "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"]

for model_name in model_names:

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0]  # 取得所有token的嵌入向量
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    def get_sentence_embedding(sentence):
        # 分詞並準備模型輸入
        encoded_input = tokenizer(sentence, return_tensors='pt', max_length=512, truncation=True)

        # 計算嵌入向量
        with torch.no_grad():
            model_output = model(**encoded_input)

        # 取平均池化作為句子向量
        attention_mask = encoded_input['attention_mask']
        sentence_embedding = mean_pooling(model_output, attention_mask)

        # 正規化
        sentence_embedding = F.normalize(sentence_embedding, p=2, dim=1)

        return sentence_embedding.squeeze()

    def cosine_similarity(a, b):
        dot_product = torch.dot(a, b)
        
        norm_a = torch.norm(a, p=2)
        norm_b = torch.norm(b, p=2)
        
        similarity = dot_product / (norm_a * norm_b)
        return similarity

    # 獲得詞嵌入
    word_embedding1 = get_sentence_embedding('張立杰教授 簡歷')
    word_embedding2 = get_sentence_embedding('規章與辦法 – 中央大學|學習與教學研究所 【行政】 1. 2. 3. 【教師】   【學生】')
    word_embedding3 = get_sentence_embedding('張立杰教授 – 中央大學|學習與教學研究所 國立中央大學|學習與教學研究所 Graduate Institute of Learning and Instruction, National Central University, Taiwan Toggle Navigation 首頁 最新消息 關於學習所 本所簡介 規章與辦法 規章與辦法 教師表格下載 學生表格下載 行政與聯絡 師資與研究 教師介紹 研究主題與方法 學生與校友 學生 出國開會心得 校友研究 課程 修業辦法 課程地圖與開課紀錄 師培中心 網路資源 教室租借 網站連結 English 張立杰教授 Li-Chieh Chang, Professor (03) 4227151 ext 33870 bchang.tw@gmail.com 個人網站 Bio/簡歷 中央大學、師培中心主任 (2020.8 ~ ) 中央大學、學習與教學研究所所長 (2020.8 ~ ) 中央大學、師資培育中心/學習與教學研究所、教授 (2019.08 ~ ) 台灣永齡希望小學中大分校負責人 (2014 ~ ) 中央大學、教務處、註冊組組長 (2018.8 ~2021.1) 中央大學、師資培育中心、主任 (2014.08 ~ 2017.07) 中央大學、師資培育中心/學習與教學研究所、副教授 (2013.08 ~2019.07) 嘉義大學、數位學習設計與管理學系、副教授 (2011.08 ~2013.07 ) 嘉義大學、教學發展中心、教學科技與學習輔導組組長 (2009.02 ~ 2012.02) 嘉義大學、數位學習設計與管理學系、助理教授 (2008.02 ~ 2011.07) 中央大學、學習科技研究中心、博士後研究 (2006.07 ~ 2008.01) 美國史丹福大學、創新學習科技中心、訪問博士後研究 (2005.08 ~ 2006.07) 中央大學、學習科技研究中心、博士後研究 (2003.10 ~ 2005.07) Teaching/教學 109-2 自然領域探究與實作 科技與教學整合專題研究 109-1 書報討論 教育實習 108-2 自然領域探究與實作 通識中心 – 科技與閱讀寫作 108-1 科技與教學整合專題研究 教育實習 107-2 教育議題專題 教學媒體與操作 107-1 科技與教學創新整合專題研究 生活科技概論 教育實習 Research/研究 計畫: 2018-2021: 社會學習網絡之自動化知識關聯分析及建構 (MOST 107-2511-H-008 -006 -MY3) 2016-2018: 關聯式設計策略：概念知識雲及其社會學習網絡之建置 (MOST 105-2511-S-008 -007 -MY2) 2013-2016: 基於教室學習網絡設計之遊戲化 Apps 及其人因分析 (MOST 102-2511-S-008 -019 -MY3) 2012: 從認知風格探討科技學習工具介入之群組式數位學習環境之影響 (MOST 101-2511-S-415-016-) 2010 – 2011: 教室中多人共用一機之競爭學習情境設計及學生喜好性分析 (99-2511-S-415-006-MY2) 2009: 使用廣域無線感測網路支援氣象科學探究活動 (98-2511-S-415-008) 2008: 教室內無線感測網路支援群體手勢活動之互動學習環境設計 (97-2511-S-415-013) 論文: 28.Shih, YA., & Chang, B. (2020). Empirical study on the effects of social network–supported group concept mapping. RPTEL 15, 23 (2020). https://doi.org/10.1186/s41039-020-00143-7 27.Shih, Y., Chang, B., & Chin, J. Y. (2020). Data-driven student homophily pattern analysis of online discussion in a social network learning environment. Journal of Computers in Education, 7, 373-394. doi: https://doi.org/10.1007/s40692-020-00160-x 26.Chan, T. W., Looi, C. K., Chang, B., Chen, W., Wong, L. H., Wong, S. L., … Chou, C. Y. (2019). IDC theory: Creation and the creation loop. RPTEL 14, 26. doi:10.1186/s41039-019-0120-5 25.Chan, T. W., Looi, C. K., Chen, W., Wong, L. H., Chang, B., Liao, C. C. Y., … Ogata, H. (2018). Interest-driven creator theory: towards a theory of learning design for Asia in the twenty-first century. Journal of Computers in Education, 5(4), 435-461. 24.Chang, B., Shih, Y. A., & Lu, F. C. (2018). Co-construction concept through cloud-based social network platform design, implementation, and evaluation. The International Review of Research in Open and Distributed Learning, 19(5), 238-253. (SSCI) doi: http://dx.doi.org/10.19173/irrodl.v19i5.3419 23.Chang, B., & Lu, F. C. (2018). Social media facilitated English prewriting activity design and evaluation. The Asia-Pacific Education Researcher, 27(1), 33-42. (SSCI) doi: 10.1007/s40299-017-0363-0 22.Yu, F. Y., Hsieh, H. T., & Chang, B. (2017). The potential of Second Life for university counseling: A comparative approach examining media features and counseling problems. Research and Practice in Technology Enhanced Learning.12:24. doi: 10.1186/s41039-017-0064-6 21.Chang, B., Chen, S., Tsai, Y. C., & Lai, M. L. (2016). The effects of task activities and gaming scales on eye and visual search performance. Computers in Human Behavior, 66, 16-25. (SSCI) 20.Yang, E. F. Y., Chang, B., Cheng, H. N. H., & Chan, T. W. (2016). Improving pupils’ mathematical communication abilities through computer-supported reciprocal peer tutoring. Educational Technology & Society, 19(3), 157-169. (SSCI) 19.Lu, F. C., & Chang, B. (2016). Role-play game enhanced English for specific purpose vocabulary acquisition framework. Educational Technology & Society, 19(2), 367-377. (SSCI) 18.Wang, J. H., Chen, S. Y., Chang, B., & Chan, T. W. (2016). From integrative peer response to game-based integrative peer response: High ability vs. low ability. Journal of Computer Assisted Learning, 32(2), 170-185. (SSCI) 17.Cheng, H. N. H., Yang, E. F. Y., Liao, C. C. Y., Chang, B., Huang, Y. C. Y., & Chan, T. W. (2015). Scaffold seeking: A reverse design of scaffolding in computer-supported word problem solving. Journal of Educational Computing Research, 53, 409-435. (SSCI) 16.Chang, B., Chen, S., & Jhan, S.N. (2015). The influences of an interactive group-based videogame: Cognitive styles vs. prior ability. Computers & Education, 88, 399-407. (SSCI) 15.Chuang, M. T., Chang, B., & Chen, H. C. (2014). Investigating reported social and affective strategy used by EFL learners in virtual and real learning environments. International Journal of Applied Linguistics & English Literature, 3(2), 27-33. 14.Chang, B., Yu, F. Y., Chen, Y. Y., & Hsieh, H. T. (2013). Thinking in an opposing position: A framework design of role-reversal pedagogy through technology. Research and Practice in Technology Enhanced Learning, 8(3), 347-362. 13.Chang, B., Chuang, M. T., & Ho, S. (2013). Understanding students’ competition preference in multiple-mice supported classroom. Educational Technology & Society, 16(1), 171-182. (SSCI) 12.Yu, F. Y., Hsieh, H. T., & Chang B. (2011). Potential of second life for psychological counseling: A comparative approach. Lecture Notes in Computer Science, 6872, 44–45. (EI) 11.Chang, B., Wang, H. Y., Peng, T. Y., & Hsu, Y. S. (2010). Development and evaluation of a city-wide wireless weather sensor network. Educational Technology & Society, 13(3), 270-280. (SSCI) 10.Elwell, M., & Chang, B. (2010, January). Mission-oriented situated second language learning in Second Life. IEEE Learning Technology Newsletter, 12(1), 62-65. 9.Chang, S. B., Lin, C. J., Ching, E., Cheng, H. N. H., Chang, B., Chen, F. C., … Chan, T. W. (2009). EduBingo: Developing a content sample for the one-to-one classroom by the content-first design approach. Educational Technology & Society, 12(3), 343-353. (SSCI) 8.Wang, H. Y., Chang, B. & Harn, C. T. (2009, July). Using mobile data logger and city-wide weather database to facilitate parent-child coupled climatology learning. IEEE Learning Technology Newsletter, 11(3), 4-6. 7.Chang, B., Wang, H. Y. & Lin, Y. S. (2009, January-April). Enhancement of mobile learning using wireless senor network. IEEE Learning Technology Newsletter, 11(1&2), 22-25. 6.Chang, B., Cheng, N. H., Deng, Y. C. & Chan, T. W. (2007). Environmental design for a structured network learning society. Computers and Education, 48(2), 234-249. (SSCI, EI) 5.Liang, J. K., Liu, T. C., Wang, H. Y., Chang, B., Deng, Y. C., Yang, J. C., Chou, C. Y., Ko, H. W., Yang, S., & Chan, T. W. (2005). A few design perspectives on one-on-one digital classroom. Journal of Computer-Assisted Learning, 21(3), 181-189. (SSCI) 4.Chang, L. J., Chou, C. Y., Chen, Z. H., & Chan, T. W. (2004). An approach to assisting teachers in building physical and network hybrid community-based learning environments: The Taiwanese experience. Journal of Educational Development, 24(4), 361-381. (SSCI) 3.Chang, L. J., Yang, J. C., Deng, Y. C., & Chan, T. W. (2003). EduXs: Multilayer educational services platforms. Computers & Education, 41(1), 1-18. (SSCI, EI) 2.Chang, L. J., Yang, J. C., Yu, F. Y., & Chan, T. W. (2003). Development and evaluation of multiple competitive activities in a synchronous quiz game system. Journal of Innovations in Education and Training International, 40(1), 16-26. (SSCI) 1.Yu, F. Y., Chang, L. J., Liu, Y. H., & Chan, T. W. (2002). Learning preferences and satisfaction of senior high school students toward different sunchronized computerized competitive modes. Journal of Computer-Assisted Learning, 18(3), 341-350. (SSCI)   桃園巿中壢區中大路300號 人文社會科學大樓LS-401室 中央大學學習與教學研究所 Phone: 886-3-4227151 # 33851 Fax: 886-3-4273371 Email: ncu3851@ncu.edu.tw 中央大學入口網站 MyNCU © 2017 學習與教學研究所. All Rights Reserved. About this site Kale, A Food Blog Theme by LyraThemes.com.')

    # 計算相似度
    similarity_1_2 = cosine_similarity(word_embedding1, word_embedding2)
    similarity_1_3 = cosine_similarity(word_embedding1, word_embedding3)
    similarity_2_3 = cosine_similarity(word_embedding2, word_embedding3)

    print(model_name)
    print(f"Similarity between word_1_2:", similarity_1_2.item())
    print(f"Similarity between word_1_3:", similarity_1_3.item())
    print(f"Similarity between word_2_3", similarity_2_3.item())

    print(len(word_embedding1))

sentence-transformers/paraphrase-mpnet-base-v2
Similarity between word_1_2: 0.5558281540870667
Similarity between word_1_3: 0.694909930229187
Similarity between word_2_3 0.5728631019592285
768
sentence-transformers/multi-qa-mpnet-base-dot-v1
Similarity between word_1_2: 0.5421507358551025
Similarity between word_1_3: 0.6456298828125
Similarity between word_2_3 0.6729167699813843
768
sentence-transformers/multi-qa-MiniLM-L6-cos-v1
Similarity between word_1_2: 0.3625856637954712
Similarity between word_1_3: 0.5489000678062439
Similarity between word_2_3 0.5522013902664185
384


tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

d:\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\chrbe\.cache\huggingface\hub\models--sentence-transformers--gtr-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/219M [00:00<?, ?B/s]

Some weights of T5Model were not initialized from the model checkpoint at sentence-transformers/gtr-t5-base and are newly initialized: ['decoder.block.0.layer.0.SelfAttention.k.weight', 'decoder.block.0.layer.0.SelfAttention.o.weight', 'decoder.block.0.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.0.layer.0.SelfAttention.v.weight', 'decoder.block.0.layer.0.layer_norm.weight', 'decoder.block.0.layer.1.EncDecAttention.k.weight', 'decoder.block.0.layer.1.EncDecAttention.o.weight', 'decoder.block.0.layer.1.EncDecAttention.q.weight', 'decoder.block.0.layer.1.EncDecAttention.v.weight', 'decoder.block.0.layer.1.layer_norm.weight', 'decoder.block.0.layer.2.DenseReluDense.wi.weight', 'decoder.block.0.layer.2.DenseReluDense.wo.weight', 'decoder.block.0.layer.2.layer_norm.weight', 'decoder.block.1.layer.0.SelfAttention.k.weight', 'decoder.block.1.layer.0.SelfAttention.o.weight', 'decoder.block.1.layer.0.SelfAttention.q.weig

ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds

graphviz.sources.Source